<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab and environment settings

Printing in colab

In [1]:
from google.colab.data_table import DataTable
DataTable.max_columns = 60

Clone repository

In [2]:
!git clone https://github.com/ncn-foreigners/job-ads-classifier.git 

Cloning into 'job-ads-classifier'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 106 (delta 35), reused 46 (delta 5), pack-reused 0
Receiving objects: 100% (106/106), 2.16 MiB | 12.23 MiB/s, done.
Resolving deltas: 100% (35/35), done.


Set-up virtual env

In [4]:
!pip install virtualenv
!virtualenv classifier
!source /content/classifier/bin/activate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 15.0 MB/s 
     |████████████████████████████████| 468 kB 66.5 MB/s 
created virtual environment CPython3.7.15.final.0-64 in 839ms
  creator CPython3Posix(dest=/content/classifier, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.2.2, setuptools==65.3.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


Upgrade pip

In [ ]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 39.7 MB/s eta 0:00:00


Download models

In [11]:
!gdown 1N2LAJyG5tZT5UjHWRUFOgia5dSPwz1eg
!unzip models-v1.0.zip
!rm -rf __MACOSX
!mv models ./job-ads-classifier/

Downloading...
From: https://drive.google.com/uc?id=1N2LAJyG5tZT5UjHWRUFOgia5dSPwz1eg
To: /content/models-v1.0.zip
100% 1.87G/1.87G [00:08<00:00, 210MB/s]


Install required packages

In [14]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!pip install -r /content/job-ads-classifier/requirements-py37-colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 582 kB 14.5 MB/s 
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.7.7
    Uninstalling pytorch-lightning-1.7.7:
      Successfully uninstalled pytorch-lightning-1.7.7


## Working with trained models

In [1]:
models_dir="/content/job-ads-classifier/models"

Load required modules

In [2]:
import sys
sys.path.insert(1, '/content/job-ads-classifier')

In [3]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from datetime import datetime
from job_offers_classifier.load_save import load_to_df, save_obj, load_obj
from job_offers_classifier.job_offers_utils import create_hierarchy, fix_class_str, remove_classes, filter_classes, remap_classes
from job_offers_classifier.job_offers_classfier import *
#from sklearn.model_selection import train_test_split
import torch
#from torchmetrics import MetricCollection, Recall, Accuracy, ConfusionMatrix

Example data -- job offers from Public Sercice

In [4]:
kprm_df=pd.read_xml("https://nabory.kprm.gov.pl/pls/serwis/app.xml", xpath = "./new/oferta")
kprm_df = kprm_df.replace("\\<\\!\\[CDATA\\[ ", "", regex=True)
kprm_df = kprm_df.replace(" \\]\\]\\>", "", regex=True)
kprm_df = kprm_df.replace(r'<[^<>]*>', '', regex=True)
kprm_df = kprm_df.replace('&nbsp;', ' ', regex=True)
kprm_df = kprm_df.replace('&dash;', ' ', regex=True)
kprm_df = kprm_df.replace('&oacute;', ' ', regex=True)
kprm_df = kprm_df.replace('\r', ' ', regex=True)
kprm_df['desc'] = kprm_df[['stanowisko', 'do_spraw', 'nazwa_firmy', 'poledodatkowelista1', 'poledodatkowelista3']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
kprm_df[["unid", "nazwa_firmy", "stanowisko", "do_spraw", "poledodatkowelista1", "poledodatkowelista3"]].head(n=2)


,unid,nazwa_firmy,stanowisko,do_spraw,poledodatkowelista1,poledodatkowelista3
0,108667,Główny Urząd Geodezji i Kartografii w Warszawie,starszy specjalista,wsparcia procesów utrzymania,"Wspiera proces planowania, przygotowania i mon...",praca przy monitorze ekranowym powyżej 4 godz...
1,108666,Główny Urząd Geodezji i Kartografii w Warszawie,specjalista,zamówień publicznych,Weryfikuje pod względem formalnym (prawnym i p...,Praca przy monitorze ekranowym powyżej 4 godz...


In [5]:
kprm_df.desc[0]

'starszy specjalista_wsparcia procesów utrzymania_Główny Urząd Geodezji i Kartografii w Warszawie_Wspiera proces planowania, przygotowania i monitorowania postępowań zakupowych i przetargowych w zakresie utrzymania i eksploatacji systemów teleinformatycznych GUGiK w tym postępowania realizowane ze środków Unijnych.Procesuje wnioski o nadanie uprawnień dostępu do systemów oraz przygotowuje dokumentację związaną z Systemem Zarządzania Bezpieczeństwem Informacji w tym zakresie.Ewidencjonuje i rejestruje działania związane z obsługą umów utrzymaniowych związanych z realizacja zadań wydziału m.in. przy użyciu narzędzia HPSM.Utrzymuje aktualną ewidencję zasobów infrastruktury teleinformatycznej będącej częścią SIG w tym informacje o gwarancjach i wsparciu technicznym.Współpracuje z podmiotami świadczącymi usługi II i III linii wsparcia dla elementów infrastruktury SIG.Współtworzy, aktualizuje i utrzymuje dokumentację środowiska teleinformatycznego GUGiK w tym w zakresie zgodności dokumentacj

### Logistic regression model using tf-df matrix

In [6]:
model_paper_linear = LinearJobOffersClassifier()
model_paper_linear.load(f"{models_dir}/linear_model")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [7]:
kprm_df_zaw1 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:01<00:00, 237.60it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1740.38it/s]


Distribution of occupancy codes

In [8]:
np.sum(kprm_df_zaw1, axis=0)

0      0.030980
1     15.750659
2    225.388809
3    168.990189
4     31.394083
5      4.069289
6      0.533869
7      2.238279
8      0.946757
9      0.636597
dtype: float32

Top 5 occupancy codes

In [9]:
kprm_df_zaw6_top5 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:03<00:00, 112.54it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1829.06it/s]


In [10]:
kprm_df_zaw6_top5.head(n=5)

,class_1,class_2,class_3,class_4,class_5,prob_1,prob_2,prob_3,prob_4,prob_5
0,251101,252902,251103,251102,252201,0.224459,0.192830,0.136211,0.079776,0.036477
1,242225,331490,242217,261103,334306,0.935973,0.015968,0.006675,0.004264,0.002557
2,121101,242217,242290,121904,242204,0.484566,0.054834,0.040809,0.021522,0.020431
3,121101,121103,121190,122106,122102,0.973492,0.012645,0.003823,0.001396,0.001201
4,441402,134907,325512,515303,341201,0.168923,0.050569,0.031137,0.015885,0.012722


### Transformer model

In [11]:
model_paper_trans = TransformerJobOffersClassifier(gpus=1, threads=2)
model_paper_trans.load(f"{models_dir}/transformer_model")

Initializing TransformerClassifier with model_name=allegro/herbert-base-cased, output_type=linear, num_labels=2905, learning_rate=1e-05, weight_decay=0.01, warmup_steps=50 ...


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
kprm_df_zaw1_trans = model_paper_trans.predict(
    X=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

Distribution of occupancy codes

In [16]:
np.sum(kprm_df_zaw1_trans, axis=0)

0      0.061671
1     22.453873
2    179.178436
3    147.483887
4     43.386158
5     10.851691
6      2.837573
7     21.614429
8     15.975781
9      6.156576
dtype: float32

Compare linear vs transformer

In [22]:
kod_linear = np.argmax(np.array(kprm_df_zaw1), axis=1)
kod_trans = np.argmax(np.array(kprm_df_zaw1_trans), axis=1)
compare=pd.DataFrame({"linear": kod_linear, "trans":kod_trans})
pd.crosstab(compare.linear, compare.trans)

trans,1,2,3,4,7
linear,,,,,
1,4,2,7,2,0
2,1,181,30,9,0
3,3,67,97,16,1
4,0,14,2,13,0
7,0,0,1,0,0


## Compare with official dictionary 

Selected occupancies from official dictionary to see which model is better.

1. 712401 -- monter izolacji budowlanych
2. 214102 -- inżynier organizacji i planowania produkcji
3. 932101 -- pakowacz ręczny


In [42]:
opisy = ["Monter dociepleń budynków - Francja, okolice Grenoble , Zagranica od ATERIMA WORK FR Monter dociepleń budynków Francja, okolice Grenoble, Zagranica Umowa o pracę tymczasową Obowiązki:montaż zewnętrznej izolacji termicznej (styropian) malowanie elewacji montaż / demontaż rusztowań niezbędnych do wykonania pracy Wymagania:doświadczenie na podobnym stanowisku dyspozycyjność do podjęcia zatrudnienia od zaraz prawo jazdy kat. B oraz mile widziany własny samochód do dyspozycji badania wysokościowe na podstawie naszego skierowania Wybranym kandydatom oferujemy:legalne zatrudnienie na terenie Francji u sprawdzonego klienta umowę o pracę tymczasową do 30 lipca z możliwością przedłużenia 12,90 EUR brutto / h (za 35 godzin tygodniowo) dodatek wyżywieniowy 10,30 EUR brutto za każdy dzień roboczy zapewnione nadgodziny płatne dodatkowo (powyżej 35 godz.: 16,12 EUR brutto/h, powyżej 43 godz.: 19,35 EUR brutto/h) zawsze terminową wypłatę wynagrodzenia składki ZUS opłacane od średniej krajowej bezpłatne zakwaterowanie częściowy zwrot kosztów podróży do Francji Co jest dla Ciebie ważne? Wiemy, że do dobrej pracy potrzebne są odpowiednie warunki. Dobrze znamy naszych kandydatów i świetnie rozumiemy, na czym szczególnie im zależy w pracy. Obowiązki:montaż zewnętrznej izolacji termicznej (styropian) malowanie elewacji montaż / demontaż rusztowań niezbędnych do wykonania pracy Wymagania:doświadczenie na podobnym stanowisku dyspozycyjność do podjęcia zatrudnienia od zaraz prawo jazdy kat. B oraz mile widziany własny samochód do dyspozycji badania wysokościowe na podstawie naszego skierowania Wybranym kandydatom oferujemy:legalne zatrudnienie na terenie Francji u sprawdzonego klienta umowę o pracę tymczasową do 30 lipca z możliwością przedłużenia 12,90 EUR brutto / h (za 35 godzin tygodniowo) dodatek wyżywieniowy 10,30 EUR brutto za każdy dzień roboczy zapewnione nadgodziny płatne dodatkowo (powyżej 35 godz.: 16,12 EUR brutto/h, powyżej 43 godz.: 19,35 EUR brutto/h) zawsze terminową wypłatę wynagrodzenia składki ZUS opłacane od średniej krajowej bezpłatne zakwaterowanie częściowy zwrot kosztów podróży do Francji",
         "Specjalista ds. planowania Twój zakres obowiązkówplanowanie produkcji zgodnie z polityką i procedurami firmyoptymalizacja i budowa efektywnych modeli planistycznych (proces ciągły)wywoływanie zakupów i rezerwacja surowców do zleceń produkcyjnychanaliza rotacji i optymalizacja poziomu zapasu surowców (proces ciągły)udział w projektach wewnętrznych optymalizujących procesy planowania i zakupówraportowanie wyników KPI w podległym obszarze Nasze wymaganiaminimum dwuletnie doświadczenie w planowaniu produkcji z wykorzystaniem systemu klasy ERPdoświadczenie w obszarze zakupów surowców i materiałów do produkcjiwykształcenie wyższego (preferowane kierunki ekonomia, inżynieria produkcji, logistyka)umiejętność analizy danych, wyciągania wniosków oraz proponowania rozwiązańznajomość języka angielskiego w stopniu komunikatywnym (B2)znajomość obsługi komputera i pakietu MS Office - w szczególności Excel na poziomie minimum średniozaawansowanym – warunek koniecznysamodzielność w działaniu i umiejętność podejmowania decyzjidobra organizacji pracy i zarządzania czasem oraz umiejętności ustalania priorytetów – warunek koniecznyMile widzianedoświadczenie we wdrażaniu systemów do planowania klasy ERPdoświadczenie w diagnozie i modelowaniu procesów produkcyjnych To oferujemystabilne zatrudnienie w oparciu o umowę o pracę w pełnym wymiarze godzinpracę w zespole nastawionym na współpracę, dzielenie się wiedzą i wspólny szacunekatrakcyjne wynagrodzenie aby docenić Twoje kompetencjepełną wyzwań pracę w dziale Zarządzania Łańcuchem Dostaw w firmie produkcyjnejmożliwość rozwoju i codziennego doskonalenia Twoich kwalifikacji zawodowychudział w projektach wewnętrznych optymalizujących nasze procesypracę od poniedziałku do piątku w godzinach od 08:00 do 16:00pakiet benefitów pozapłacowych w tym ubezpieczenie na życiepakiet socjalny w ramach ZFŚS",
         "Pakowacz Zakres obowiązków:- pakowanie towaru- etykietowanie towaru- ważenie towaruWymagania:Wymagania konieczne: Wykształcenie: podstawowe, brakWynagrodzenie brutto: od 2 800 PLNRodzaj umowy:Umowa o pracę na czas określonyLiczba wolnych miejsc pracy:10, dla niepełnosprawnych 1Sposób aplikowania:bezpośrednio do pracodawcyPracodawca:Zakład przemysłu mięsnego biernacki sp. z o.o.Profil działalności pracodawcy:przetwarzanie i konserwowanie mięsa,z wyłączeniem mięsa z drobiuMiejsce pracy: Dworcowa 47 D, 63-200 Golina, powiat: jarociński, woj: wielkopolskieKontakt:Preferowane formy kontaktu: telefon, e-mailNumer telefonu:62 747 09 27Wymagane dokumenty:CVData rozpoczęcia pracy:13.07.2021Data ważności ogłoszenia:30.09.2021Dodane przez:Powiatowy Urząd Pracy w JarocinieIdentyfikator oferty:StPr/21/0506"]

In [43]:
trans3=kprm_df_zaw1_trans = model_paper_trans.predict(X=list(opisy), output_level="last", format="dataframe", top_k=1)
lin3=kprm_df_zaw1_trans = model_paper_linear.predict(X_text=list(opisy), output_level="last", format="dataframe", top_k=1)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

Processing X ...
Processing text ...


100%|██████████| 3/3 [00:00<00:00, 141.92it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 3/3 [00:00<00:00, 1252.65it/s]


In [44]:
results = pd.concat([trans3,lin3], axis = 1)
results["correct"] = pd.Series(["712401", "214102", "932101"])
results

,class_1,prob_1,class_1,prob_1,correct
0,712301,0.528665,712401,0.843104,712401
1,214102,0.706504,214102,0.840713,214102
2,932101,0.990134,932101,0.646111,932101
